# Задача 12. Экспериментальное исследование алгоритмов решения задачи достижимости с контекстно-свободными ограничениями
# Автор: [kznts9v-1lya](https://github.com/kznts9v-1lya)


## Введение
Пусть имеется размеченный ориентированный граф $G$. Будем интерпретировать его как конечный автомат. Таким образом, считаем, что граф $G$ (его представление в виде КА) задает регулярный язык $L_G$.

Пусть нам задана КС-грамматика $CFG$, задающая язык ограничений $L_{CFG}$.

Рассмотрим задачу $CFPQ = \{(v_i, v_j)|\exists\pi:w(v_i\pi v_j)\in L, v_i\in V_S, v_j\in V_F\}$, где $L = L_G\cap L_{CFG}$

Для языка $L_{CFG}$ строим рекурсивный автомат $-$ в нашем случае являющийся конечным автоматом над смешанным алфавитом грамматики.

## Постановка задачи
Задача посвящена анализу производительности различных алгоритмов решения задачи достижимости с контекстно-свободными ограничениями: алгоритма Хеллингса, матричного алгоритма, тензорного алгоритма. В ходе анализа необходимо
- Сравнить производительности реализаций различных алгоритмов для одной аппаратной платформы (CPU, GPU) и определить наиболее производительный алгоритм в рамках одной платформы.
- Сравнить производительности различных реализаций алгоритмов для разных платформ (CPU vs. GPU) и определить наиболее производительную реализацию.
- Сравнить производительности CFPQ алгоритмов со специализированными RPQ алгоритмами в задаче RPQ анализа и определить необходимость существования решения для частных случаев.

## Описание работы

- [x] Используя [pyCuBool](https://pypi.org/project/pycubool/) реализовать матричный алгоритм решения задачи достижимости с КС ограничениями. Аналогично тому, как это было сделано в домашней работе 10.
- [x] Используя [pyCuBool](https://pypi.org/project/pycubool/) реализовать тензорный алгоритм решения задачи достижимости с КС ограничениями. Аналогично тому, как это было сделано в домашней работе 11.
- [x] Подключить реализацию алгоритма Хеллингса из работы 9, матричного из 10 и тензорного из 11.
- [x] Сформировать набор данных.
  - [x] Выбрать все графы из раздела [RDF](https://jetbrains-research.github.io/CFPQ_Data/dataset/RDF.html).
  - [x] Запросы к RDF состоят из следующих блоков
    - Запросы из работы 5. Так как регулярные языки --- это подмножество контекстно-свободных, то решения, рассматриваемые в данной работе должны справляться с ними. Взять необходимо ровно те запросы, которые использовали именно вы. При этом, обратите внимание на то, что для разных алгоритмов нужен разный формат входа. Например, для тензорного грамматику из регулярного выражения можно получить просто сделав это выражение правой частью единственного правила.
    - Три классических запроса для анализа иерархии.
      - ```S -> subClassOf_r S subClassOf | type_r S type | subClassOf_r subClassOf | type_r type ```
      - ```S -> subClassOf_r S subClassOf | subClassOf ```
      - ```S -> broaderTransitive S broaderTransitive_r | broaderTransitive broaderTransitive_r ```
  - [x] Выбрать графы bzip, gzip, ls, init, drivers, arch. Это графы, построенные по реальным программам и предназначенные для решения задач статического анализа.
  - [x] Для статического анализа использовать единственный запрос. Обратите внимание, что он записан в максимально общем виде и непосредственно грамматика будет зависеть от алгоритма, который будет исследоваться.
    - ```
       M -> d_r V d
       V -> (M? a_r)∗ M? (a M?)∗
      ```
- [x] С использованием сформированного набора данных провести сравнение производительности следующих решений
  - алгоритм Хеллингса
  - матричный алгоритм, реализованный с использованием sciPy
  - тензорный алгоритм, реализованный с использованием sciPy
  - матричный алгоритм, реализованный с использованием pyCuBool
  - тензорный алгоритм, реализованный с использованием pyCuBool
  - алгоритм выполнения регулярных запросов, реализованный с использованием pyCuBool (работа 5). Здесь только для RDF и регулярных запросов.
  - алгоритм выполнения регулярных запросов, реализованный с использованием sciPy. Здесь только для RDF и регулярных запросов.
- [x] Оформить результаты экспериментов
- [x] Провести анализ результатов
- [x] Сделать notebook доступным по ссылке (обязательно выдать права на комментирование проверяющему)

## Настройка проекта и зависимостей

In [1]:
!git clone https://github.com/kznts9v-1lya/formal-lang-course.git &> /dev/null

In [2]:
%cd formal-lang-course

/content/formal-lang-course


In [3]:
!git checkout task12-cfpq-research &> /dev/null

In [50]:
%mkdir -p benchmark
%mkdir benchmark/scipy_cfpq_matrix
%mkdir benchmark/pycubool_cfpq_matrix
%mkdir benchmark/scipy_cfpq_hellings
%mkdir benchmark/scipy_cfpq_tensor
%mkdir benchmark/pycubool_cfpq_tensor

In [5]:
!pip install -r requirements.txt &> /dev/null

In [4]:
!pip install pycubool &> /dev/null

## Описание набора данных

### Графы

Датасет состоит из графов и запросов, представленных контекстно-свободными грамматиками над метками соответствующих графов.

Для проведения эксперимета использовались RDF графы, представленные в библиотеке [CFPQ_Data](https://github.com/JetBrains-Research/CFPQ_Data), кроме графов *taxonomy*, *taxonomy_hierarchy* (которые не поместились в ОЗУ) и *go_hierarchy* (который имеет всего одну метку).

In [6]:
from project.graph_tools import get_from_dataset

In [7]:
graph_names = [
             'atom_primitive', 
             'biomedical_mesure_primitive', 
             'core', 
             'eclass_514en', 
             'enzyme', 
             'foaf', 
             'funding', 
             'generations', 
             'geospecies', 
             'go', 
             'go_hierarchy', 
             'pathways', 
             'people_pets', 
             'pizza', 
             'skos', 
             'travel', 
             'univ_bench', 
             'wine'
] 

#### Загрузка данных

In [8]:
graphs = {}

for name in graph_names:
  graph = get_from_dataset(name, True)
  graphs[name] = graph

Loading...: 100%|██████████| 273/273 [00:00<00:00, 123574.90it/s]
file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.
file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.
Loading...: 100%|██████████| 1839/1839 [00:00<00:00, 88816.12it/s]


#### Характеристики данных

In [9]:
import pandas as pd

data = {name: [graph.description.nodes, graph.description.edges, len(graph.description.edge_labels)] for name, graph in graphs.items()}
columns = ['Nodes', 'Edges', 'Labels']

graph_descriptions = pd.DataFrame.from_dict(data, orient='index', columns=columns)

graph_descriptions

,Nodes,Edges,Labels
atom_primitive,291,425,17
biomedical_mesure_primitive,341,459,10
core,1323,2752,31
eclass_514en,239111,360248,10
enzyme,48815,86543,14
foaf,256,631,15
funding,778,1086,20
generations,129,273,17
geospecies,450609,2201532,165
go,582929,1437437,47


Граф go_hierarchy имеет всего 1 метку, поэтому не будем рассматривать этот набор данных:

In [10]:
graph_names.remove('go_hierarchy')
graphs.pop('go_hierarchy', None)

Разобьем графы на большие и небольшие для удобства постановки экспериментов:

In [11]:
large_graph_names = ['eclass_514en', 'enzyme', 'geospecies', 'go']
small_graph_names = list(set(graph_names) - set(large_graph_names))

large_graphs = {name: graphs[name] for name in large_graph_names}
small_graphs = {name: graphs[name] for name in small_graph_names}

### Запросы

- Регулярные запросы (так как регулярные языки $-$ это подмножество контекстно-свободных). К каждому графу $G$ отправлялись регулярные запросы следующего вида:

  - $(l_1|l_2)*l_3$,

  - $(l_1|l_2)^+l_3^*$,

  - $l_1l_2l_3(l_4|l_1)^*$,

  - $(l_1|l_2|l_3)^*$,
  
  где $l_1, l_2, l_3, l_4$ – первые 4 метки графа $G$. Все вершины графа помечались как стартовые и финальные.

- Три классических запроса для анализа иерархии.

  - ```
  S -> subClassOf_r S subClassOf | type_r S type | subClassOf_r subClassOf | type_r type 
  ```

  - ```
  S -> subClassOf_r S subClassOf | subClassOf 
  ```

  - ```
  S -> broaderTransitive S broaderTransitive_r | broaderTransitive broaderTransitive_r 
  ```
  
- Для статического анализа:

  - ```
       M -> d_r V d
       V -> (M? a_r)* M? (a M?)*
    ```


#### Генерация запросов

In [12]:
from project.grammar_tools import get_cfg_from_text


def create_first_regex(labels):
  """
  (l1|l2)*l3
  """

  r1 = str(labels[0])
  r2 = str(labels[1])
  r3 = str(labels[2])

  return get_cfg_from_text(f'S -> A {r3}\n A -> {r1} A \n A -> {r2} A \n A -> epsilon \n')


def create_second_regex(labels):
  """
  (l1|l2)+l3*
  """

  r1 = str(labels[0]) 
  r2 = str(labels[1])
  r3 = str(labels[2])

  return get_cfg_from_text(f'S -> С A B\n A -> {r1} A \n A -> {r2} A \n A -> epsilon \n B -> {r3} B \n B -> epsilon \n C -> {r1} | {r2}')


def create_third_regex(labels):
  """
  l1 l2 l3 (l4|l1)*
  """

  r1 = str(labels[0])
  r2 = str(labels[1])
  r3 = str(labels[2])
  r4 = str(labels[3])

  return get_cfg_from_text(f'S -> {r1} {r2} {r3} A \n A -> {r1} A \n A -> {r4} A \n A -> epsilon')


def create_fourth_regex(labels):
  """
  (l1|l2|l3)*
  """

  r1 = str(labels[0])
  r2 = str(labels[1])
  r3 = str(labels[2])

  return get_cfg_from_text(f'S -> epsilon \n S -> {r1} S \n S -> {r2} S \n S -> {r3} S')


def get_label_by_substr(labels, substr):
  for label in labels:
    if label.count(substr):
      return label

  return substr


def add_reverse_labels(graph, labels, label):
  e = list(graph.edges(data="label", default=''))

  for (u, v, l) in e:
    if l.count(label):
      graph.add_edge(v, u, label=l + '_r')


def create_first_hierarchy(graph, labels):
  subClassOf = get_label_by_substr(labels, 'subClassOf')
  add_reverse_labels(graph, labels, 'subClassOf')
  subClassOf_r = subClassOf + '_r'
  type_ = get_label_by_substr(labels, 'type')
  add_reverse_labels(graph, labels, 'type')
  type_r = type_ + '_r'

  return get_cfg_from_text(f'S -> {subClassOf_r} S {subClassOf} | {type_r} S {type_} | {subClassOf_r} {subClassOf} | {type_r} {type_}')


def create_second_hierarchy(graph, labels):
  subClassOf = get_label_by_substr(labels, 'subClassOf')
  add_reverse_labels(graph, labels, 'subClassOf')
  subClassOf_r = subClassOf + '_r'

  return get_cfg_from_text(f'S -> {subClassOf_r} S {subClassOf} | {subClassOf}')


def create_third_hierarchy(graph, labels):
  broaderTransitive = get_label_by_substr(labels, 'broaderTransitive')
  add_reverse_labels(graph, labels, 'broaderTransitive')
  broaderTransitive_r = broaderTransitive + '_r'

  return get_cfg_from_text(f'S -> {broaderTransitive} S {broaderTransitive_r} | {broaderTransitive} {broaderTransitive_r}')


def create_queries(graph, labels):
  return [
      create_first_regex(labels),
      create_second_regex(labels),
      create_third_regex(labels),
      create_fourth_regex(labels),
      create_first_hierarchy(graph, labels),
      create_second_hierarchy(graph, labels),
      create_third_hierarchy(graph, labels)
  ]

## Описание эксперимента

### Оборудование
Эксперимент проводился в облачной среде *Google Colab*. Алгоритм на основе матриц из `scipy` запускался на CPU, на основе `pycubool` – на GPU. 


In [13]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


CPU:

In [14]:
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s)'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep 'Thread(s) per core'
!lscpu | grep "MHz"

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Socket(s):           1
Core(s) per socket:  1
Thread(s) per core:  2
CPU MHz:             2299.998


GPU:

In [15]:
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Mon Dec 13 20:09:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

RAM:

In [16]:
!grep MemTotal /proc/meminfo

MemTotal:       13302924 kB


### Реализация алгоритмов



#### scipy (CPU)

In [17]:
from project.path_querying_tools import (
    hellings_context_free_path_querying,
    matrix_context_free_path_querying,
    tensor_context_free_path_querying
)

#### pyCuBool (GPU)

In [ ]:
import pycubool as cb

from networkx import MultiDiGraph
from pyformlang.cfg import CFG
from project.grammar_tools import get_cnf_from_cfg

def matrix_cb(cfg: CFG, graph: MultiDiGraph) -> set:
    n = graph.number_of_nodes()
    if n == 0:
        return set()

    result = {}
    term_prods = set()
    nonterm_prods = set()

    if cfg.generate_epsilon():
        m = cb.Matrix.empty(shape=(n, n))
        for i in range(n):
            m[i, i] = True
        result[cfg.start_symbol.value] = m

    cfg = cfg_to_normal_form(cfg)

    for prod in cfg.productions:
        if len(prod.body) == 1:
            term_prods.add(prod)
        elif len(prod.body) == 2:
            nonterm_prods.add(prod)

    for u, v, edge_data in graph.edges(data=True):
        for prod in term_prods:
            if prod.body[0].value == edge_data["label"]:
                m = result.get(prod.head.value, cb.Matrix.empty(shape=(n, n)))
                m[u, v] = True
                result[prod.head.value] = m

    changing = True
    while changing:
        changing = False
        for p in nonterm_prods:
            m = result.get(p.head.value, cb.Matrix.empty(shape=(n, n)))
            old_nnz = set(m.to_list())
            m = m.ewiseadd(result.get(p.body[0].value, cb.Matrix.empty(shape=(n, n))).mxm(
                    result.get(p.body[1].value, cb.Matrix.empty(shape=(n, n)))
                )
            )
            new_nnz = set(m.to_list())
            result[p.head.value] = m
            changing = max(changing, old_nnz != new_nnz)

    return {
        (u, variable, v)
        for variable, matrix in result.items()
        for u, v in matrix.to_list()
    }


### Окружение

Вспомогательные функции эксперимента.

In [18]:
import csv
import signal
import time

from tqdm import tqdm

In [19]:
def write_measurements(path, rows):
  fieldnames = ['Query', 'CFPQ_size', 'Time(s)']

  with open(path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

In [20]:
def signal_handler(signum, frame):
    raise Exception("Timed out!")

signal.signal(signal.SIGALRM, signal_handler)

<Handlers.SIG_DFL: 0>

In [21]:
def measure_cfpq_time(graph, queries, fabm, num_runs, timeouts=None):
    rows = []

    for _ in range(num_runs):
      for i, query in enumerate(queries):
        cfpq_ans = start_time = end_time = None

        if timeouts:
            signal.alarm(int(timeouts[i] + 1))

        try:
            start_time = time.time()
            if fabm == 'cfpq_hellings':
              cfpq_ans = hellings_context_free_path_querying(graph, query)
            elif fabm == 'cfpq_matrix':
              cfpq_ans = matrix_context_free_path_querying(graph, query)
            elif fabm == 'cfpq_tensor':
              cfpq_ans = tensor_context_free_path_querying(graph, query)
            elif fabm == 'cfpq_matrix_cb':
              cfpq_ans = cb_matrix_context_free_path_querying(graph, query)
            elif fabm == 'cfpq_tensor_cb':
              cfpq_ans = cb_tensor_context_free_path_querying(graph, query)
            end_time = time.time()
        except Exception:
            end_time = time.time()

        rows.append({
            'Time(s)': end_time - start_time, 
            'CFPQ_size': len(cfpq_ans) if cfpq_ans else 'NaN', 
            'Query': i
            })
        
    return rows

In [22]:
def measure_algs_performance(graphs, num_runs, algos, timeouts=None):
  for algo_name, algo, path in algos:
    print(f'Measuring performance of algorithm {algo} based on {algo_name} ...')

    for graph_name, graph in tqdm(graphs.items()):
      edge_labels = list(graph.description.edge_labels)

      measurements = measure_cfpq_time(
          graph.graph, 
          create_queries(graph.graph, edge_labels), 
          algo, 
          num_runs=num_runs,
          timeouts=timeouts
          )
      
      write_measurements(f'{path}/{graph_name}.csv', measurements)

In [23]:
def create_df(graph_names, algos):
  dfs = []

  for algo_name, algo, path in algos:
    for graph_name in graph_names:
      df = pd.read_csv(f'{path}/{graph_name}.csv')
      df.insert(0, 'Algorithm', algo_name + '_' + algo)
      df.insert(0, 'Graph', graph_name)
      dfs.append(df)

  return pd.concat(dfs, ignore_index=True)

In [24]:
def show_barplots(df, title):
  fig, axes = plt.subplots(7, 2, figsize=(18, 40))
  fig.subplots_adjust(left=0.05, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.5)
  fig.suptitle(title, fontsize=25)
  fig.delaxes(axes[6, 1])

  for i, name in enumerate(small_graph_names):
    sns.barplot(ax=axes[i // 2, i % 2], x="Query", y="Time(s)", hue='Algorithm', data=df[df['Graph'] == name]).set_title(name)

  plt.show();

## Проведение эксперимента

In [25]:
scipy_algos = [
      ('scipy', 'cfpq_hellings', 'benchmark/scipy_cfpq_hellings'),
      ('scipy', 'cfpq_matrix', 'benchmark/scipy_cfpq_matrix'),
      ('scipy', 'cfpq_tensor', 'benchmark/scipy_cfpq_tensor'),
]

In [26]:
measure_algs_performance(small_graphs, num_runs=10, algos=scipy_algos)

Measuring performance of algorithm cfpq_hellings based on scipy ...


100%|██████████| 13/13 [14:57<00:00, 69.07s/it]


Measuring performance of algorithm cfpq_matrix based on scipy ...


100%|██████████| 13/13 [00:40<00:00,  3.12s/it]


Measuring performance of algorithm cfpq_tensor based on scipy ...


 31%|███       | 4/13 [41:22<1:33:05, 620.56s/it]


KeyboardInterrupt: ignored

In [28]:
df_sm_scipy = pd.merge(create_df(small_graph_names, scipy_algos), graph_description, right_index=True, left_on='Graph')
show_barplots(df_sm_scipy, 'Производительность алгоритмов с scipy на небольших графах')

FileNotFoundError: ignored